# Estimasi Intensitas Siklon Tropis Menggunakan Deep Convolutional Neural Network - Bagian 3

**Daftar Isi :**

- [Memahami kelemahan(<i>drawbacks</i>) dari solusi yang ada](#Understanding-the-drawbacks)
- [Penyelesaian](#Working-out-the-solution)
    - [Augmentasi Data](#Data-Augmentation)
- [Melatih Model](#Training-the-Model-with-Data-Augmentation)

**Setelah menyelesaikan <i>nobteook</i> ini diharapkan peserta dapat :**

- Paham bagaimana cara meningkatkan kinerja Model yang sudah ada.
- Augmentasi Data.
- Bereksperimen dengan Hyperparameters.

# Memahami Kekurangan (<i>drawbacks</i>)

```python3
Sederhananya, baik buruknya model machine learning bergantung dengan kualitas data yang dimasukkan
```

Kita telah mencapai akurasi hampir ~85% berjalan dengan 4 <i>epochs</i>. Sekarang kita akan mencoba meningkatkan akurasi dengan melihat lebih dekat pada dataset dan gambar. Kita dapat mengamati hal berikut dari buku catatan kita sebelumnya :

Hal pertama yang akan kita perhatikan dari penghitungan kategori adalah bahwa jumlah gambar per kategori sangat tidak seragam dengan rasio TC:H5 **lebih besar dari 1:20**, Ketidakseimbangan ini dapat membiaskan visi model CNN kita karena memprediksi salah pada kelas minoritas tidak akan berdampak banyak pada model karena kontribusi kelas kurang dari 5% dari kumpulan data.

Hal yang sama dapat ditunjukkan juga oleh <i>heatmap</i> yang kita peroleh di <i>notebook</i> sebelumnya : Perhatikan Sebagian besar Kelas dengan data yang lebih tinggi diprediksi dengan benar dan kelas minoritas lebih salah diprediksi daripada kelas lainnya
![alt_text](images/heatmap.png)


Mari kita lihat sekarang bagaimana kita memecahkan masalah itu menggunakan augmentasi data :

## Penyelesaian

## Augmentasi Data

Untuk mengurangi ketidakseragaman, kita akan membalik dan memutar gambar untuk mengkompensasi kekurangan data untuk kelas dengan sampel yang lebih sedikit:

![alt text](images/augment.png)


Kita akan menggunakan OpenCV untuk membalik dan memutar gambar.

``` python
cv2.flip(img,0)
cv2.flip(img,1)
cv2.warpAffine(img, cv2.getRotationMatrix2D(center, 90, 1.0), (h, w))
cv2.warpAffine(img, cv2.getRotationMatrix2D(center, 180, 1.0), (w, h))
cv2.warpAffine(img, cv2.getRotationMatrix2D(center, 270, 1.0), (h, w))
```

Ada cara lain untuk mengatasi ketidakseimbangan data seperti Class weightage, Oversampling, SMOTE dll..

# Melatih Model menggunakan Augmentasi Data

Kita membuat fungsi baru bernama `augmentation(name,category,filenames,labels,i)` dan di sini kita menambahkan lebih banyak sampel ke Kategori yang memiliki data tidak seimbang.

In [ ]:
import sys
sys.path.append('/workspace/python/source_code')
# Import Fungsi utilitas
from utils import * 
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"


# Mendeklarasikan Fungsi Augmentasi
def augmentation(name,category,filenames,labels,i):
    # Important Constants
    file_path = "Dataset/Aug/"
    images = []
    (h, w) = (232,232)
    center = (w / 2, h / 2)
    angle90 = 90
    angle180 = 180
    angle270 = 270
    scale = 1.0
    img = load_image(name , interpolation = cv2.INTER_LINEAR)
    
    
    if category == 0 :
        images.append(cv2.flip(img,0))
    elif category == 1 :
        pass
    elif category == 2 :
        pass
    elif category == 3 :
        pass
    elif category == 4 :
        pass
    elif category == 5 :
        pass
    elif category == 6 :
        pass
    elif category == 7 :
        images.append(cv2.flip(img,0))
        
        
    for j in range(len(images)):
        cv2.imwrite(file_path+str(i+j)+'.jpeg',images[j])
        filenames.append(file_path+str(i+j)+'.jpeg')
        labels.append(category)
    i = i + len(images)
    return i

##### Kita meneruskan fungsi ini ke fungsi `load_dataset()` kita untuk menghasilkan augmentasi ini.

Harap tunggu beberapa menit sambil memperbesar gambar.

In [ ]:
filenames,labels = load_dataset(augment_fn = augmentation)

In [ ]:
# Setel Ukuran set Validasi
val_filenames , val_labels = make_test_set(filenames,labels,val=0.1)

In [ ]:
#Buat set Uji dan Latih
test = 0.1
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(filenames, labels, test_size=test, random_state=1)

In [ ]:
import tensorflow as tf
y_train = tf.one_hot(y_train,depth=8)
y_test = tf.one_hot(y_test,depth=8)
val_labels = tf.one_hot(val_labels,depth=8)

In [ ]:
# Jadikan Dataset kompatibel dengan Tensorflow Data Pipelining.
train,test,val = make_dataset((x_train,y_train,128),(x_test,y_test,32),(val_filenames,val_labels,32))

# Model yang dijelaskan dalam Makalah  :

Sekarang kita akan menggunakan model yang dijelaskan di makalah untuk mengevaluasi keakuratannya pada kumpulan data baru.

![alt_text](images/model.png)

In [ ]:
import numpy as np
tf.random.set_seed(1337)

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten ,Dropout, MaxPooling2D
from tensorflow.keras import backend as K 

#Reset Graphs dan Buat ulang Sequential model

K.clear_session()
model = Sequential()
#Convolution Layers

model.add(Conv2D(64, kernel_size=10,strides=3, activation='relu', input_shape=(232,232,3)))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(256, kernel_size=5,strides=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(288, kernel_size=3,strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=1))
model.add(Conv2D(272, kernel_size=3,strides=1,padding='same',activation='relu'))
model.add(Conv2D(256, kernel_size=3,strides=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Dropout(0.5))
model.add(Flatten())

#Linear Layers 

model.add(Dense(3584,activation='relu'))
model.add(Dense(2048,activation='relu'))
model.add(Dense(8, activation='softmax'))

# Print Model Summary

model.summary()

In [ ]:
import functools
# Include Top-2 Accuracy Metrics 
top2_acc = functools.partial(tensorflow.keras.metrics.top_k_categorical_accuracy, k=2)
top2_acc.__name__ = 'top2_acc'

#Deklarasi jumlah Epochs
epochs = 4

#Dikarenakan Melatih model kita dari awal akan memakan waktu lama
#Jadi kita akan memuat model yang dilatih sebagian untuk mempercepat proses
model.load_weights("trained_16.h5")

# Optimizer
sgd = tensorflow.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9)


#Compile Model dengan Loss Function , Optimizer dan Metrics
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, 
              optimizer=sgd,
              metrics=['accuracy',top2_acc])

# Latih Model
trained_model = model.fit(train,
          epochs=epochs,
          verbose=1,
          validation_data=val)

# Test Model dengan data Validasi
score = model.evaluate(test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


### Visualisasi

In [ ]:
import matplotlib.pyplot as plt
f = plt.figure(figsize=(15,5))
ax = f.add_subplot(121)
ax.plot(trained_model.history['accuracy'])
ax.plot(trained_model.history['val_accuracy'])
ax.set_title('Model Accuracy')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
ax.legend(['Train', 'Val'])

ax2 = f.add_subplot(122)
ax2.plot(trained_model.history['loss'])
ax2.plot(trained_model.history['val_loss'])
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Val'],loc= 'upper left')

plt.show()

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
import pandas as pd

#Pembutan heatmap menggunakan confusion matrix

pred = model.predict(val)
p = np.argmax(pred, axis=1)
y_valid = np.argmax(val_labels, axis=1, out=None)
results = confusion_matrix(y_valid, p) 
classes=['NC','TD','TC','H1','H3','H3','H4','H5']
df_cm = pd.DataFrame(results, index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (15,15))

sn.heatmap(df_cm, annot=True, cmap="Blues")

Simpan Model dan bobot terlatih untuk penggunaan di masa mendatang :

In [ ]:
#Menyimpan Model
model.save('cyc_pred.h5')

## Licensing
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).